In [174]:
#stuff

# Purpose of this Notebook

In [175]:
#Number of reviews to put into data frame of good reviews
NUMB_OF_DOCS1 = 1000

#Number of reviews to put into data frame of bad reviews
NUMB_OF_DOCS2=100

## Problem Statement

Product developers often want to know this about their products:

* For products that people don't like, what is it about the products they don't like? Maybe if we know this we could improve the product by fixing this unliked product features
* For products that people love, what is it about the products that result in people loving them? Maybe if we expanded on these loved features, we could get new customers or better retain current customers

## Proposed Solution

Topic modeling and clustering(?) that identifies the features of products that people love and somehow rank them so that a product manaer can have a priority list of things that could be addressed to get more love
Topic modeling and clustering(?) that identifies the features of products that people don't like and somehow rank them so that a product manaer can have a priority list of things that could be fixed to hopefully turn the disliked product into a liked product.


# Module Import

In [209]:
import datetime
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



import re

#to speed up pandas operands
from pandarallel import pandarallel

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import nltk
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import pyLDAvis
import pyLDAvis.sklearn

#gensim
from gensim import corpora, models, similarities, matutils



# NLP Pipeline



## Text Preprocessing
Essentially will do this:

```

(input text) -->[clean(remove text I don't want)]--> [tokenization] --> [lemmatization] --> [Next_Stages]

```

In [177]:
reviews_df = pd.read_csv("../Data/Reviews.csv")

In [178]:
reviews_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Going to create a dataframe of bad reviews and one of good reviews to process separately

In [179]:
good_reviews_df = reviews_df[ reviews_df["Score"] >= 4 ]
bad_reviews_df = reviews_df[ reviews_df["Score"] <= 2 ]

In [180]:
if (NUMB_OF_DOCS1 > good_reviews_df.shape[0]) or (NUMB_OF_DOCS1 < 0):
    NUMB_OF_DOCS1 = good_reviews_df.shape[0]


if (NUMB_OF_DOCS2 > bad_reviews_df.shape[0]) or (NUMB_OF_DOCS2 < 0):
    NUMB_OF_DOCS2 = bad_reviews_df.shape[0]

In [181]:
stuff =list(good_reviews_df.Text.head(NUMB_OF_DOCS1))

In [182]:
#Removing HTML from the text
stuff = [re.sub("<.*?>","",item) for item in stuff]

In [183]:
#remove punctuation
stuff = [  re.sub("[\.|\!|\?|\,|\;|\:|\&|\(|\)|\-]","",mystring) for mystring in stuff  ]

**Tokenize**

In [184]:
stuff_tokenized= []

stuff_tokenized = [word_tokenize(review) for review in stuff]

In [185]:
#stuff_tokenized[1]

In [186]:
#for i,item in enumerate(stuff_tokenized):
#    if i == 10:
#        break
#    print(i,"-->",item,"\n")

In [187]:
#lemmatize each word in stuff
def calc_partofspeech(word):
    """generates the pos encoding that's accepted by WordNetLemmatizer.lemmatize()"""
    #print("word = ",word)
    [(output_word, output_pos)] = pos_tag(word_tokenize(word))
    
    #decode output_pos to the pos required by the lemmatizer
    
    if "JJ" in output_pos:
        pos = "a"
    elif "RB" in output_pos:
        pos = "r"
    elif "VB" in output_pos:
        pos = "v"
    else:
        pos = "n"
    
    return pos




In [188]:
#for i,item in enumerate(stuff):
#   if i == 10:
#        break
#   print(i,"-->",item,"\n")

lemmatizing to cut down on the size of the dimensions of the eventual vectorized word matrix without losing meaning of the text

In [189]:
wordNetLemmatizer = WordNetLemmatizer()

stuff_lemmatized = []

for mydoc in stuff_tokenized:
    temp_mydoc_lemmatized = []
    for myword in mydoc:
        temp_word_lemmatized = wordNetLemmatizer.lemmatize(myword,calc_partofspeech(myword))
        temp_mydoc_lemmatized.append(temp_word_lemmatized)

    stuff_lemmatized.append(temp_mydoc_lemmatized)
    
#stuff_lemmatized = [wordNetLemmatizer.lemmatize(myword,calc_partofspeech(myword)) for mydoc in stuff_tokenized for myword in mydoc]



In [190]:
#Test code doesn't need to be run
#myword = "running"
#myword_lemmed = wordNetLemmatizer.lemmatize(myword,calc_partofspeech(myword))
#myword_lemmed = wordNetLemmatizer.lemmatize(myword,"v")

#print("myword_lemmed = ",myword_lemmed)
#print("POS for myword = ",calc_partofspeech(myword))

In [191]:
#stuff_lemmatized[0]

In [192]:
# I only want adjectives and adverbs sent to the vectorizer
#stuff_pos_filtered = [mylemword for mylemdoc in stuff_lemmatized for mylemword in mylemdoc if ("JJ" in pos_tag(word_tokenize(mylemword))[0][1]) or ("RB" in pos_tag(word_tokenize(mylemword))[0][1])]

In [193]:
#stuff_pos_filtered[6]

In [198]:
# I only want adjectives and adverbs sent to the vectorizer

stuff_pos_filtered = []

for mylemdoc in stuff_lemmatized:
    
    temp_pos_filtered_doc = ""
    
    for mylemword in mylemdoc:
        if ("JJ" in pos_tag(word_tokenize(mylemword))[0][1]) or ("RB" in pos_tag(word_tokenize(mylemword))[0][1]):
            temp_pos_filtered_doc = temp_pos_filtered_doc + mylemword + " "
            
    stuff_pos_filtered.append(temp_pos_filtered_doc.rstrip())


In [199]:
stuff_pos_filtered[0:5]

['several good more well well most',
 'few tiny then liberally tiny Not too very highly familiar',
 'great wide very',
 'very enjoyable many only there too much just not particular last only',
 'great very soft individually well together expensive highly']

In [200]:
len(stuff_pos_filtered)

1000

## Word Vectorization / Parsing

Will do the following here:

```

(lemmatized text)-->[WordVectorization(dropping stop word)] --> [POS tagging] --> [collect Adjectives]


```


In [201]:
vectorizer = CountVectorizer(stop_words="english")
docTermMatrixCountVec = vectorizer.fit_transform(stuff_pos_filtered)

In [202]:
len(vectorizer.get_feature_names())

678

In [203]:
docTermMatrixCountVec.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [204]:
docTermMatrixCountVec.shape

(1000, 678)

In [205]:

docTermMatrixCountVec_df = pd.DataFrame(docTermMatrixCountVec.toarray(), index=stuff_pos_filtered, columns=vectorizer.get_feature_names())

In [206]:
docTermMatrixCountVec_df.head()

,able,absolutely,acceptable,accessable,active,actual,actually,addictedfirst,addictive,addictively,...,wintergreen,wish,wonderful,wonderfully,workable,worried,worth,wrong,wrongobviously,young
several good more well well most,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
few tiny then liberally tiny Not too very highly familiar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
great wide very,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
very enjoyable many only there too much just not particular last only,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
great very soft individually well together expensive highly,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [207]:
LD_Allocator = LatentDirichletAllocation(n_components=5, random_state=0)
LD_Allocator.fit(docTermMatrixCountVec)

LatentDirichletAllocation(n_components=5, random_state=0)

## Topics via LDA

## LDA on "Good" Reviews

In [210]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LD_Allocator, docTermMatrixCountVec, vectorizer)

/home/magreen/anaconda3_2020_07/envs/metisgpulinux2venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.075135  0.039413       1        1  26.375063
1     -0.092016  0.071130       2        1  26.143931
0     -0.068258  0.019015       3        1  20.655793
2      0.006246 -0.181655       4        1  15.506893
3      0.229162  0.052098       5        1  11.318320, topic_info=         Term        Freq       Total Category  logprob  loglift
482    really  144.000000  144.000000  Default  30.0000  30.0000
249      good  288.000000  288.000000  Default  29.0000  29.0000
548     small   74.000000   74.000000  Default  28.0000  28.0000
254     green   40.000000   40.000000  Default  27.0000  27.0000
279       hot   65.000000   65.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
339      long    5.198019   39.015725   Topic5  -4.7294   0.1631
564    strong    4.702979   37.052680   Topic5  -4.8295   0.1146
6    actually    4.482708   39.829215   Topic5  -4.8774  -0.0056
367   natural    4.262264   46.705090   Topic5  -4.9279  -0.2153
648   usually    3.899497   23.942554   Topic5  -5.0168   0.3639

[328 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  0.248720         able
0         3  0.373080         able
0         4  0.041453         able
0         5  0.331627         able
1         1  0.443394   absolutely
...     ...       ...          ...
665       1  0.758858         wide
668       4  0.754299  wintergreen
673       1  0.683405      worried
673       2  0.227802      worried
677       2  0.850758        young

[477 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 4])